# LSTM Model


In this section we will implement the LSTM Model (Long-Short-Term Memory). 

The model uses a stacked LSTM architecture with dropout regularization for multivariate time series forecasting. It consists of:

- **LSTM layers**: Process sequential input features (value, temporal features, and one-hot encodings) through multiple recurrent layers
- **Dropout**: Applied between LSTM layers and after the final output to prevent overfitting
- **Fully Connected layer**: Maps the last hidden state to a single prediction value

The model takes sequences of multiple features at each timestep and outputs a single forecasted value by leveraging the LSTM's ability to capture long-term temporal dependencies.



In [ ]:
import torch
import torch.nn as nn

## Model

In [ ]:
class LSTMForecaster(nn.Module):
    """
    LSTM model for MULTIVARIATE time series forecasting.
    Architecture: LSTM -> Dropout -> LSTM -> Dropout -> Fully Connected
    Takes multiple input features at each timestep.
    """
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2):
        """
        Args:
            input_size: Number of input features (Value + year + month + one-hot)
            hidden_size: LSTM hidden dimension
            num_layers: Number of LSTM layers
            dropout: Dropout rate
        """
        super(LSTMForecaster, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        
        # LSTM forward pass
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Take the output from the last time step
        last_output = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_size)
        
        # Apply dropout
        out = self.dropout(last_output)
        
        # Fully connected layer
        out = self.fc(out)  # Shape: (batch_size, 1)
        
        return out

In [ ]:
!cd .. && python ./neuralts/main.py --model LSTM --epochs 25 --batch-size 128 --learning-rate 0.0005